# Transfer Learning for Animal Image Classification

In this homework assignment, you will develop and refine a neural network model for classifying animal images from a provided dataset containing five classes: "alpaca", "bear", "elephant", "leopard", and "zebra". The dataset (alpaca_and_others_dataset.zip) is available on MS Teams under the path General/Class Materials/datasets. The primary goal is to improve upon a baseline model through experimentation with different architectures, hyperparameters, and techniques like transfer learning and data augmentation.

## Baseline Model

Select a neural network architecture and a set of hyperparameters to train the network. You can split the dataset as train and validation, no need for a separate test set. Refer to the resulting model as "baseline model." Note that this baseline model's performance may be low but we should be able to observe improvement (decrease in loss, increase in validation accuracy) after the training.

## Controlled Experiments

Apply changes that can potentially improve the baseline model to obtain several candidate models each of which differs from the baseline model by a single modification, for example:

- Transfer learning rather than training from scratch (mandatory)
- Using a different architecture
- Data augmentation

Make sure that you perform "controlled experiments." Clearly specify what is changed in each candidate model. Train them for the same number of epochs, and compare all the results by plotting their training and validation losses with respect to epochs. You can smooth the curves if needed. Share your findings and comments.

## Proposed Model

Propose one new model by observing the previous results. You can combine multiple changes together (e.g. a combination of transfer learning and data augmentation if you observe that both of these improve the performance). Briefly share your conclusions as a list of facts and comments.

## Repeatability

In your homework ensure repeatability of your results by implementing the following practices:

- **Setting Random Seeds:** Initialize random number generator seeds for ALL randomness modules directly or indirectly used (e.g. `random`, `tensorflow.random` etc.) to ensure reproducibility of random processes.

- **Environment Reproducibility:** At the beginning of your code specify the versions of dependencies (e.g., TensorFlow) used in your environment. This ensures that the same environment can be recreated to reproduce the results.

- **Deterministic Processes:** Avoid using processes whose results rely on external factors such as operating system or computation power. For example, ensure deterministic file listing (e.g., use `sorted(os.listdir())` instead of `os.listdir())` and fixed number of epochs rather than using a predetermined time budget such as 1 hour for training.

## Rules

You are expected to use TensorFlow-Keras library and submit a single Jupyter/Colab Notebook. Please make sure to show each step clearly in your notebook without leaving any room for doubt and without any exception. Please add visualizations, analysis, or comments if necessary. Please note that performing transfer learning in at least one of the candidate models is mandatory. There are many TF-Keras transfer learning examples online. You can examine them. There must be sufficient amount of individual work in the solution: Selection of models, selection of layers for transfer learning, data augmentation options, selection of hyperparameters, regularization options etc. must differ from the available solutions. Groupwork is NOT allowed. You will prepare and submit homeworks individually.

For making the homework evaluation easier, please make sure the submitted notebook includes separate sections for different tasks and displays all the outputs without the need for running the code!



# Animal Image Classification

### Import necessary libraries

In [ ]:
import os
import random
import shutil

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

: 

In [ ]:
# Print versions for reproducibility
print(f"TensorFlow version: {tf.__version__}")
print(f"NumPy version: {np.__version__}")

In [ ]:
# Set random seeds for reproducibility
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
# Define paths
dataset_dir = "/c:/Users/murat/Desktop/term_8/CENG506/HWs/hw2/dataset"
output_dir = "/c:/Users/murat/Desktop/term_8/CENG506/HWs/hw2/splitted_dataset"

In [ ]:
# Create train and validation directories
train_dir = os.path.join(output_dir, "train")
val_dir = os.path.join(output_dir, "val")

# Make output directories
os.makedirs(output_dir, exist_ok=True)
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

In [ ]:
# Split ratio
val_split = 0.2

# Process each class
classes = ["alpaca", "bear", "elephant", "leopard", "zebra"]

for cls in classes:
    # Create class subdirectories
    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)
    
    # Get all images for this class
    class_path = os.path.join(dataset_dir, cls)
    images = sorted([img for img in os.listdir(class_path) if img.endswith(('.jpg', '.JPEG', '.png'))])
    
    # Split into train and validation
    train_images, val_images = train_test_split(images, test_size=val_split, random_state=SEED)
    
    # Copy images to respective directories
    for img in train_images:
        src = os.path.join(class_path, img)
        dst = os.path.join(train_dir, cls, img)
        shutil.copy(src, dst)
    
    for img in val_images:
        src = os.path.join(class_path, img)
        dst = os.path.join(val_dir, cls, img)
        shutil.copy(src, dst)
    
    print(f"Class {cls}: {len(train_images)} training images, {len(val_images)} validation images")

print("\nDataset split complete!")

In [ ]:
# Check class distribution
print("\nClass distribution:")
for split in ["train", "val"]:
    print(f"\n{split.capitalize()} set:")
    split_path = os.path.join(output_dir, split)
    for cls in classes:
        cls_path = os.path.join(split_path, cls)
        num_images = len(os.listdir(cls_path))
        print(f"  {cls}: {num_images} images")

In [ ]:
# Define image parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Create data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

In [ ]:
def create_baseline_model():
    model = models.Sequential([
        # First Convolutional Block
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        layers.MaxPooling2D((2, 2)),
        
        # Second Convolutional Block
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Third Convolutional Block
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Flatten and Dense Layers
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(5, activation='softmax')  # 5 classes
    ])
    
    return model

# Create and compile the model
baseline_model = create_baseline_model()
baseline_model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Display model summary
baseline_model.summary()

In [ ]:
# Training parameters
EPOCHS = 20

# Train the model
history = baseline_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE
)

In [ ]:
def plot_training_history(history):
    # Plot training & validation accuracy
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    # Plot training & validation loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    plt.tight_layout()
    plt.show()

# Plot the training history
plot_training_history(history)